# Review Radar - FastAPI Backend 

## 1. Imports and Dependencies

In [25]:
# Core FastAPI and web framework imports
from fastapi import FastAPI, HTTPException
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel, Field
from typing import List, Dict, Any
import uvicorn

# Machine Learning and NLP imports
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
import torch

# Utility imports
import logging
import asyncio
import json
from datetime import datetime

print("All imports successful!")
print(f"PyTorch CUDA available: {torch.cuda.is_available()}")
print(f"PyTorch version: {torch.__version__}")

All imports successful!
PyTorch CUDA available: True
PyTorch version: 2.8.0+cu126


## 2. Configure Logging

In [26]:
# Configure logging for better debugging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Test logging
logger.info("Logging configured successfully!")
print("Logger ready for use")

2025-09-27 20:59:26,710 - __main__ - INFO - Logging configured successfully!


Logger ready for use


## 3. Data Models (Pydantic)

In [27]:
# Pydantic models for request/response validation

class ReviewAnalysisRequest(BaseModel):
    """Request model for sentiment analysis"""
    reviews: List[str] = Field(
        ..., 
        description="List of review texts to analyze", 
        min_items=1,
        example=["This product is amazing!", "Terrible quality, very disappointed"]
    )

class ReviewSentiment(BaseModel):
    """Individual review sentiment result"""
    review: str
    sentiment: str
    confidence: float

class AnalysisResponse(BaseModel):
    """Complete analysis response with summary"""
    results: List[ReviewSentiment]
    summary: Dict[str, int]
    total_reviews: int
    processing_time_ms: float = 0.0

# Test the models
test_request = ReviewAnalysisRequest(reviews=["Great product!", "Not good at all"])
print("Pydantic models created successfully!")
print(f"Test request: {test_request.model_dump()}")

Pydantic models created successfully!
Test request: {'reviews': ['Great product!', 'Not good at all']}


## 4. Initialize Sentiment Analysis Model

In [28]:
# Global sentiment analyzer (will be initialized once)
sentiment_analyzer = None

def initialize_sentiment_model():
    """
    Initialize the sentiment analysis model on startup
    Using DistilBERT fine-tuned on Stanford Sentiment Treebank (SST-2)
    """
    global sentiment_analyzer
    
    try:
        logger.info("Loading sentiment analysis model...")
        
        # Load the specific model requested
        model_name = "distilbert-base-uncased-finetuned-sst-2-english"
        
        # Initialize the pipeline with the model
        sentiment_analyzer = pipeline(
            "sentiment-analysis",
            model=model_name,
            tokenizer=model_name,
            return_all_scores=True,  # Get confidence scores
            device=0 if torch.cuda.is_available() else -1  # Use GPU if available
        )
        
        device_info = 'GPU (CUDA)' if torch.cuda.is_available() else 'CPU'
        logger.info(f"Model loaded successfully! Using device: {device_info}")
        
        # Test the model with a sample text
        test_result = sentiment_analyzer("This is a great product!")
        logger.info(f"Model test successful: {test_result}")
        
        return True
        
    except Exception as e:
        logger.error(f"Failed to load sentiment model: {str(e)}")
        return False

# Initialize the model
success = initialize_sentiment_model()
print(f"Model initialization: {'SUCCESS' if success else 'FAILED'}")

2025-09-27 20:59:26,735 - __main__ - INFO - Loading sentiment analysis model...
Device set to use cuda:0
Device set to use cuda:0
2025-09-27 20:59:28,785 - __main__ - INFO - Model loaded successfully! Using device: GPU (CUDA)
2025-09-27 20:59:28,785 - __main__ - INFO - Model loaded successfully! Using device: GPU (CUDA)
2025-09-27 20:59:28,816 - __main__ - INFO - Model test successful: [[{'label': 'NEGATIVE', 'score': 0.00012437114492058754}, {'label': 'POSITIVE', 'score': 0.9998756647109985}]]
2025-09-27 20:59:28,816 - __main__ - INFO - Model test successful: [[{'label': 'NEGATIVE', 'score': 0.00012437114492058754}, {'label': 'POSITIVE', 'score': 0.9998756647109985}]]


Model initialization: SUCCESS


## 5. Helper Functions

In [33]:
# Enhanced sentiment analysis with neutral detection
def analyze_reviews_batch_enhanced(reviews: List[str], neutral_threshold: float = 0.75) -> List[ReviewSentiment]:
    """
    Enhanced sentiment analysis with neutral detection based on confidence threshold
    If confidence is below threshold, classify as neutral
    """
    if not sentiment_analyzer:
        raise Exception("Sentiment model not initialized")
    
    try:
        logger.info(f"Analyzing {len(reviews)} reviews with neutral detection...")
        start_time = datetime.now()
        
        # Get predictions for all reviews
        predictions = sentiment_analyzer(reviews)
        
        results = []
        for i, (review, prediction) in enumerate(zip(reviews, predictions)):
            # Get the highest confidence prediction
            best_prediction = max(prediction, key=lambda x: x['score'])
            
            # Apply neutral threshold logic
            if best_prediction['score'] < neutral_threshold:
                sentiment = "Neutral"  # Low confidence = neutral
                confidence = round(1 - best_prediction['score'], 4)  # Invert confidence for neutral
            else:
                sentiment = map_sentiment_label(best_prediction['label'])
                confidence = round(best_prediction['score'], 4)
            
            results.append(ReviewSentiment(
                review=review,
                sentiment=sentiment,
                confidence=confidence
            ))
            
            logger.debug(f"Review {i+1}: {sentiment} (confidence: {confidence})")
        
        processing_time = (datetime.now() - start_time).total_seconds() * 1000
        logger.info(f"Processing completed in {processing_time:.2f}ms")
        
        return results
        
    except Exception as e:
        logger.error(f"Error during enhanced sentiment analysis: {str(e)}")
        raise

# Test the enhanced version
def test_enhanced_analysis():
    """Test the enhanced analysis with neutral detection"""
    test_reviews_enhanced = [
        "This product is absolutely amazing! Best purchase ever!",  # Clearly positive
        "Terrible quality, broke after one day. Very disappointed.",  # Clearly negative  
        "It's okay, nothing special but does the job.",  # Should be neutral
        "The product works fine.",  # Should be neutral
        "Not bad, not great, just average.",  # Should be neutral
        "Love it! Highly recommend to everyone!",  # Clearly positive
        "Worst product I've ever bought. Complete waste of money."  # Clearly negative
    ]
    
    print("Testing ENHANCED sentiment analysis...")
    print("="*60)
    
    # Test with different thresholds
    thresholds = [0.75, 0.85, 0.95, 0.99]
    
    for threshold in thresholds:
        print(f"\nTesting with neutral threshold: {threshold}")
        print("-" * 40)
        
        try:
            results = analyze_reviews_batch_enhanced(test_reviews_enhanced, threshold)
            
            # Show results
            for i, result in enumerate(results, 1):
                print(f"{i}. {result.sentiment} ({result.confidence:.3f}) - {result.review[:50]}...")
            
            # Summary
            summary = calculate_summary(results)
            print(f"\nSummary: Pos:{summary['Positive']}, Neg:{summary['Negative']}, Neu:{summary['Neutral']}")
            
        except Exception as e:
            print(f" Error: {str(e)}")
            print(f"Error: {str(e)}")
print(" Enhanced sentiment analysis with neutral detection ready!")
print("Enhanced sentiment analysis with neutral detection ready!")
print("Run test_enhanced_analysis() to see the improved results!")

 Enhanced sentiment analysis with neutral detection ready!
Enhanced sentiment analysis with neutral detection ready!
Run test_enhanced_analysis() to see the improved results!


In [ ]:
# Advanced neutral detection using keyword patterns and sentiment scores
def analyze_reviews_batch_smart_neutral(reviews: List[str]) -> List[ReviewSentiment]:
    """
    Smart neutral detection using keyword patterns + confidence analysis
    Better approach for detecting truly neutral reviews
    """
    if not sentiment_analyzer:
        raise Exception("Sentiment model not initialized")
    
    # Neutral indicators - words/phrases that suggest neutral sentiment
    neutral_keywords = {
        # Lukewarm positive
        'okay', 'ok', 'fine', 'decent', 'acceptable', 'average', 'alright', 'fair',
        # Balanced expressions
        'nothing special', 'does the job', 'works as expected', 'as advertised',
        'not bad', 'not great', 'could be better', 'could be worse',
        # Neutral qualifiers
        'standard', 'basic', 'ordinary', 'typical', 'normal', 'regular',
        'so-so', 'meh', 'whatever', 'middle of the road'
    }
    
    # Strong sentiment indicators (should override neutral detection)
    strong_positive = {'amazing', 'fantastic', 'excellent', 'perfect', 'love', 'best', 'wonderful', 'incredible', 'outstanding'}
    strong_negative = {'terrible', 'awful', 'horrible', 'worst', 'hate', 'disgusting', 'useless', 'garbage', 'waste'}
    
    try:
        logger.info(f"Analyzing {len(reviews)} reviews with smart neutral detection...")
        start_time = datetime.now()
        
        # Get predictions for all reviews
        predictions = sentiment_analyzer(reviews)
        
        results = []
        for i, (review, prediction) in enumerate(zip(reviews, predictions)):
            review_lower = review.lower()
            
            # Get the highest confidence prediction
            best_prediction = max(prediction, key=lambda x: x['score'])
            original_sentiment = map_sentiment_label(best_prediction['label'])
            original_confidence = round(best_prediction['score'], 4)
            
            # Check for strong sentiment indicators first
            has_strong_positive = any(word in review_lower for word in strong_positive)
            has_strong_negative = any(word in review_lower for word in strong_negative)
            
            # Check for neutral indicators
            neutral_score = sum(1 for keyword in neutral_keywords if keyword in review_lower)
            
            # Decision logic
            if has_strong_positive or has_strong_negative:
                # Strong sentiment words override neutral detection
                sentiment = original_sentiment
                confidence = original_confidence
            elif neutral_score >= 1:  # Found neutral keywords
                # Additional checks for truly neutral reviews
                if original_confidence < 0.9 or neutral_score >= 2:
                    sentiment = "Neutral"
                    confidence = round(0.7 + (neutral_score * 0.1), 4)  # Base confidence + keyword bonus
                else:
                    # High confidence with neutral words - keep original but reduce confidence
                    sentiment = original_sentiment
                    confidence = round(original_confidence * 0.8, 4)
            else:
                # No neutral indicators - use original prediction
                sentiment = original_sentiment
                confidence = original_confidence
            
            results.append(ReviewSentiment(
                review=review,
                sentiment=sentiment,
                confidence=confidence
            ))
            
            logger.debug(f"Review {i+1}: {sentiment} ({confidence}) - Neutral keywords: {neutral_score}")
        
        processing_time = (datetime.now() - start_time).total_seconds() * 1000
        logger.info(f"Smart processing completed in {processing_time:.2f}ms")
        
        return results
        
    except Exception as e:
        logger.error(f"Error during smart sentiment analysis: {str(e)}")
        raise

# Test the smart neutral detection
def test_smart_neutral_analysis():
    """Test the smart neutral detection approach"""
    test_reviews_smart = [
        "This product is absolutely amazing! Best purchase ever!",  # Strong positive
        "Terrible quality, broke after one day. Very disappointed.",  # Strong negative  
        "It's okay, nothing special but does the job.",  # Neutral (keywords: okay, nothing special, does the job)
        "The product works fine.",  # Neutral (keyword: fine)
        "Not bad, not great, just average.",  # Neutral (keywords: not bad, not great, average)
        "Love it! Highly recommend to everyone!",  # Strong positive
        "Worst product I've ever bought. Complete waste of money.",  # Strong negative
        "Decent quality for the price, nothing extraordinary.",  # Neutral (keywords: decent, nothing)
        "Works as advertised, pretty standard stuff.",  # Neutral (keywords: as advertised, standard)
        "Meh, it's alright I guess.",  # Neutral (keywords: meh, alright)
    ]
    
    print("Testing SMART neutral detection approach...")
    print("="*65)
    
    try:
        results = analyze_reviews_batch_smart_neutral(test_reviews_smart)
        
        # Show results with more detail
        for i, result in enumerate(results, 1):
            print(f"{i:2d}. {result.sentiment:8s} ({result.confidence:.3f}) - {result.review}")
        
        # Summary
        summary = calculate_summary(results)
        print(f"\nSummary: Positive: {summary['Positive']}, Negative: {summary['Negative']}, Neutral: {summary['Neutral']}")
        print(f"Total: {sum(summary.values())} reviews")
        
        # Show improvement
        neutral_percentage = (summary['Neutral'] / sum(summary.values())) * 100
        print(f"Neutral detection rate: {neutral_percentage:.1f}%")
        
    except Exception as e:
        print(f"Error: {str(e)}")

print("Smart neutral detection with keyword analysis ready!")
print("Run test_smart_neutral_analysis() to see much better neutral detection!")

In [35]:
# Test the smart neutral detection
print("Testing Smart Neutral Detection")
print("=" * 50)

# Run the smart test
test_smart_neutral_analysis()

Testing Smart Neutral Detection


NameError: name 'test_smart_neutral_analysis' is not defined

In [34]:

# Test the enhanced neutral detection
print("Testing Enhanced Sentiment Analysis with Neutral Detection")
print("=" * 70)

# Run the enhanced test
test_enhanced_analysis()

2025-09-27 21:00:51,540 - __main__ - INFO - Analyzing 7 reviews with neutral detection...


Testing Enhanced Sentiment Analysis with Neutral Detection
Testing ENHANCED sentiment analysis...

Testing with neutral threshold: 0.75
----------------------------------------


2025-09-27 21:00:52,424 - __main__ - INFO - Processing completed in 883.48ms
2025-09-27 21:00:52,426 - __main__ - INFO - Analyzing 7 reviews with neutral detection...
2025-09-27 21:00:52,426 - __main__ - INFO - Analyzing 7 reviews with neutral detection...
2025-09-27 21:00:52,611 - __main__ - INFO - Processing completed in 184.38ms
2025-09-27 21:00:52,612 - __main__ - INFO - Analyzing 7 reviews with neutral detection...
2025-09-27 21:00:52,611 - __main__ - INFO - Processing completed in 184.38ms
2025-09-27 21:00:52,612 - __main__ - INFO - Analyzing 7 reviews with neutral detection...


1. Positive (1.000) - This product is absolutely amazing! Best purchase ...
2. Negative (1.000) - Terrible quality, broke after one day. Very disapp...
3. Positive (1.000) - It's okay, nothing special but does the job....
4. Positive (1.000) - The product works fine....
5. Negative (0.895) - Not bad, not great, just average....
6. Positive (1.000) - Love it! Highly recommend to everyone!...
7. Negative (1.000) - Worst product I've ever bought. Complete waste of ...

Summary: Pos:4, Neg:3, Neu:0

Testing with neutral threshold: 0.85
----------------------------------------
1. Positive (1.000) - This product is absolutely amazing! Best purchase ...
2. Negative (1.000) - Terrible quality, broke after one day. Very disapp...
3. Positive (1.000) - It's okay, nothing special but does the job....
4. Positive (1.000) - The product works fine....
5. Negative (0.895) - Not bad, not great, just average....
6. Positive (1.000) - Love it! Highly recommend to everyone!...
7. Negative (1.000) - Worst

2025-09-27 21:00:52,700 - __main__ - INFO - Processing completed in 87.14ms
2025-09-27 21:00:52,701 - __main__ - INFO - Analyzing 7 reviews with neutral detection...
2025-09-27 21:00:52,701 - __main__ - INFO - Analyzing 7 reviews with neutral detection...
2025-09-27 21:00:52,793 - __main__ - INFO - Processing completed in 89.88ms
2025-09-27 21:00:52,793 - __main__ - INFO - Processing completed in 89.88ms


1. Positive (1.000) - This product is absolutely amazing! Best purchase ...
2. Negative (1.000) - Terrible quality, broke after one day. Very disapp...
3. Positive (1.000) - It's okay, nothing special but does the job....
4. Positive (1.000) - The product works fine....
5. Neutral (0.105) - Not bad, not great, just average....
6. Positive (1.000) - Love it! Highly recommend to everyone!...
7. Negative (1.000) - Worst product I've ever bought. Complete waste of ...

Summary: Pos:4, Neg:2, Neu:1

Testing with neutral threshold: 0.99
----------------------------------------
1. Positive (1.000) - This product is absolutely amazing! Best purchase ...
2. Negative (1.000) - Terrible quality, broke after one day. Very disapp...
3. Positive (1.000) - It's okay, nothing special but does the job....
4. Positive (1.000) - The product works fine....
5. Neutral (0.105) - Not bad, not great, just average....
6. Positive (1.000) - Love it! Highly recommend to everyone!...
7. Negative (1.000) - Worst p

In [31]:
def map_sentiment_label(label: str) -> str:
    """
    Map model output labels to standardized sentiment labels
    DistilBERT SST-2 outputs: POSITIVE, NEGATIVE
    We map to: Positive, Negative, Neutral (though SST-2 doesn't have neutral)
    """
    label_mapping = {
        "POSITIVE": "Positive",
        "NEGATIVE": "Negative",
        "NEUTRAL": "Neutral"  # For future models that support neutral
    }
    return label_mapping.get(label.upper(), label)

def analyze_reviews_batch(reviews: List[str]) -> List[ReviewSentiment]:
    """
    Analyze sentiment for a batch of reviews
    Returns structured results with confidence scores
    """
    if not sentiment_analyzer:
        raise Exception("Sentiment model not initialized")
    
    try:
        # Process all reviews in batch for efficiency
        logger.info(f"Analyzing {len(reviews)} reviews...")
        start_time = datetime.now()
        
        # Get predictions for all reviews
        predictions = sentiment_analyzer(reviews)
        
        results = []
        for i, (review, prediction) in enumerate(zip(reviews, predictions)):
            # Get the highest confidence prediction
            best_prediction = max(prediction, key=lambda x: x['score'])
            
            sentiment = map_sentiment_label(best_prediction['label'])
            confidence = round(best_prediction['score'], 4)
            
            results.append(ReviewSentiment(
                review=review,
                sentiment=sentiment,
                confidence=confidence
            ))
            
            logger.debug(f"Review {i+1}: {sentiment} (confidence: {confidence})")
        
        processing_time = (datetime.now() - start_time).total_seconds() * 1000
        logger.info(f"Processing completed in {processing_time:.2f}ms")
        
        return results
        
    except Exception as e:
        logger.error(f"Error during sentiment analysis: {str(e)}")
        raise

def calculate_summary(results: List[ReviewSentiment]) -> Dict[str, int]:
    """
    Calculate aggregate counts of sentiment labels
    """
    summary = {"Positive": 0, "Negative": 0, "Neutral": 0}
    
    for result in results:
        if result.sentiment in summary:
            summary[result.sentiment] += 1
    
    return summary

print("Helper functions defined successfully!")

Helper functions defined successfully!
